In [1]:
import tensorflow as tf

In [9]:
# Fixes bad convolution
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [71]:
s = tf.io.read_file('day9.txt')
s = tf.strings.split(s, '\n')
s = tf.strings.to_number(s, tf.float64)

from itertools import combinations

def generate_onehots(n, k=2):
    return tf.stack(
        [tf.scatter_nd(tf.constant(comb)[:,None], tf.ones(k, tf.float64), (n,)) for comb in combinations(range(n), k)],
        0
    )

@tf.function(experimental_compile=True, experimental_relax_shapes=True)
def solve_xla(s, n, k):
    onehots = generate_onehots(n, k)

    res = tf.squeeze(tf.map_fn(
        lambda i: onehots @ tf.slice(s, [i], [n])[:,None],
        tf.range(0, s.shape[0]-n),
        fn_output_signature=tf.TensorSpec([onehots.shape[0], 1], tf.float64)
    ))

    res = ~tf.reduce_any(res == s[n:, None], 1)
    return res

@tf.function
def solve1(s, n, k):
    res = solve_xla(s, n, k)
    return s[n+tf.where(res)[0][0]]

n = 25
k = 2

%timeit solve1(s, n, k)
solve1(s, n, k)

16.1 ms ± 1.39 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


<tf.Tensor: shape=(), dtype=float64, numpy=257342611.0>

In [72]:
def do_conv(s_conv, k, x):
    res = tf.nn.conv1d(s_conv, tf.ones((k, 1, 1), tf.float64), stride=1, padding='VALID')
    if tf.reduce_any(res == x):
        idx = tf.where(res == x)[0][1]
        rnge = s[idx:idx+k]

        return tf.reduce_min(rnge) + tf.reduce_max(rnge)
    
    return tf.constant(0, tf.float64)

@tf.function
def solve2(s, x):   
    return tf.foldr(
        lambda acc, k: acc + do_conv(s[None,:,None], k, x),
        tf.range(2, s.shape[0], dtype=tf.int64),
        tf.constant(0, tf.float64)
    )


%timeit solve2(s, solve1(s, n, k))

418 ms ± 4.76 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [202]:
s

<tf.Tensor: shape=(20,), dtype=float64, numpy=
array([ 35.,  20.,  15.,  25.,  47.,  40.,  62.,  55.,  65.,  95., 102.,
       117., 150., 182., 127., 219., 299., 277., 309., 576.])>

In [11]:
import datetime
# Set up logging.
stamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = 'logs/day8_graph/%s' % stamp

# Bracket the function call with
# tf.summary.trace_on() and tf.summary.trace_export().
tf.profiler.experimental.start(logdir)
solve2()
tf.profiler.experimental.stop()